# Задание 1
Напишите функцию, которая классифицирует фильмы из материалов занятия по следующим правилам:
  - оценка 2 и меньше - низкий рейтинг
  - оценка 4 и меньше - средний рейтинг
  - оценка 4.5 и 5 - высокий рейтинг

Результат классификации запишите в столбец class


In [2]:
import pandas as pd

In [5]:
rating = pd.read_csv('ratings.csv')
rating.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [6]:
def rating_class(data):
    if data <= 2:
        return 'low'
    elif data <= 4:
        return 'medium'
    else:
        return 'hight'

rating['class'] = rating['rating'].apply(rating_class)
rating.sample(8)

,userId,movieId,rating,timestamp,class
42650,288,2761,4.0,1050509853,medium
3043,20,2054,2.5,1054036164,medium
76772,480,5902,4.5,1179160509,hight
80982,511,96737,3.0,1522156019,medium
41974,287,355,0.5,1110231294,low
98372,606,34437,4.0,1183640232,medium
2689,19,2153,3.0,965710092,medium
30876,216,4002,3.0,975212110,medium


# Задание 2
Используем файл keywords.csv.

Необходимо написать гео-классификатор, который каждой строке сможет выставить географическую принадлежность определенному региону. Т. е. если поисковый запрос содержит название города региона, то в столбце 'region' пишется название этого региона. Если поисковый запрос не содержит названия города, то ставим 'undefined'.

Правила распределения по регионам Центр, Северо-Запад и Дальний Восток:
```
geo_data = {

    'Центр': ['москва', 'тула', 'ярославль'],

    'Северо-Запад': ['петербург', 'псков', 'мурманск'],

    'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']

}
```
Результат классификации запишите в отдельный столбец region.


In [3]:
keywords = pd.read_csv('keywords.csv')

def region(data):
    '''создание столбца регион для датафрейма ключевых слов поиска'''
    geo_data = {'Центр': ['москва', 'тула', 'ярославль'],
           'Северо-Запад': ['петербург', 'псков', 'мурманск'],
           'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']}
    for geo in geo_data:
        for i in geo_data[geo]:
            if i in data:
                return geo
    return 'undefined'

keywords.head()    

,keyword,shows
0,вк,64292779
1,одноклассники,63810309
2,порно,41747114
3,ютуб,39995567
4,вконтакте,21014195


In [4]:
keywords['region'] = keywords['keyword'].apply(region)
 
res = keywords.region.value_counts() # Проверка
res 

undefined         99253
Центр               387
Северо-Запад        266
Дальний Восток       94
Name: region, dtype: int64

# Задание 3 (бонусное)
Есть мнение, что "раньше снимали настоящее кино, не то что сейчас". Ваша задача проверить это утверждение, используя файлы с рейтингами фильмов из прошлого домашнего занятия (файл ratings.csv из базы https://grouplens.org/datasets/movielens). Т. е. проверить верно ли, что с ростом года выпуска фильма его средний рейтинг становится ниже.

При этом мы не будем затрагивать субьективные факторы выставления этих рейтингов, а пройдемся по следующему алгоритму:

1. В переменную years запишите список из всех годов с 1950 по 2010.

2. Напишите функцию production_year, которая каждой строке из названия фильма выставляет год выпуска. Не все названия фильмов содержат год выпуска в одинаковом формате, поэтому используйте следующий алгоритм:
  - для каждой строки пройдите по всем годам списка years
  - если номер года присутствует в названии фильма, то функция возвращает этот год как год выпуска
  - если ни один из номеров года списка years не встретился в названии фильма, то возвращается 1900 год

3. Запишите год выпуска фильма по алгоритму пункта 2 в новый столбец 'year'

4. Посчитайте средний рейтинг всех фильмов для каждого значения столбца 'year' и отсортируйте результат по убыванию рейтинга


In [8]:
rating = pd.read_csv('ratings.csv')
movies = pd.read_csv('movies.csv')
print(rating.head()) #То ли ТЗ устарело, но в файле рейтингов нет названия и года выпуска. Зато они есть в файле movies

   userId  movieId  rating  timestamp
0       1        1     4.0  964982703
1       1        3     4.0  964981247
2       1        6     4.0  964982224
3       1       47     5.0  964983815
4       1       50     5.0  964982931


In [9]:
years = tuple(str(i) for i in range (1950, 2011))

def movie_year(data):
    '''Выделяет год из названия фильма'''
    for year in years:
        if year in data:
            return year
    return 1900

movies['year'] = movies['title'].apply(movie_year) #выделили год 
myear = movies[['movieId', 'year']]

myear.head()

,movieId,year
0,1,1995
1,2,1995
2,3,1995
3,4,1995
4,5,1995


In [19]:
rating_year = rating.merge(myear) #склеили две таблицы. Не могу придумать как выполнить задание без склейки
rating_year.head()

,userId,movieId,rating,timestamp,year
0,1,1,4.0,964982703,1995
1,5,1,4.0,847434962,1995
2,7,1,4.5,1106635946,1995
3,15,1,2.5,1510577970,1995
4,17,1,4.5,1305696483,1995


In [21]:
rating_sort = rating_year.groupby('year').mean()
rating_sort.sort_values(by = 'rating', ascending=False).head(10) 

,userId,movieId,rating,timestamp
year,,,,
1957,315.865116,3012.213953,4.039535,1.178092e+09
1954,315.735294,2426.022059,4.009191,1.159430e+09
1962,328.343511,4645.015267,3.969466,1.145154e+09
1952,356.708333,2687.718750,3.953125,1.156213e+09
1972,321.437500,4742.921196,3.944293,1.182165e+09
1964,321.351064,4900.795213,3.940160,1.180442e+09
1974,319.420601,3556.620172,3.935622,1.172035e+09
1967,312.167979,4951.223097,3.922572,1.162747e+09
1975,329.667190,3207.897959,3.879121,1.167952e+09


да, действительно раньше было лучше, трава зеленее, небо голубее, вот и фильмы лучше на 0,6 пунктов.